## Import Libraries

In [1]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pyperclip

import pandas as pd
import time

### Clipboard copy

In [2]:
def clipboard_input(driver, user_input):
    # save existing clipboard data
    existing_clipboard = pyperclip.paste()

    # copy user_input to clipboard. then paste it
    pyperclip.copy(user_input)
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()

    # recover the original clipboard data
    pyperclip.copy(existing_clipboard)  
    time.sleep(1)

### Load user info

In [3]:
def load_user_info():
    f = open('user_info.txt', 'r')
    user_info = f.readlines()
    f.close()
    
    return user_info[0][:-1], user_info[1][:-1]

### Naver Login

In [4]:
def login_naver(driver, user_id, user_pw):
    # naver login page
    driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://m.naver.com/aside/')

    # input id 
    driver.find_element_by_xpath('//*[@id="id"]').click()
    clipboard_input(driver, user_id)

    # input pw
    driver.find_element_by_xpath('//*[@id="pw"]').click()
    clipboard_input(driver, user_pw)

    # click login btn
    driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
    time.sleep(1)

    # click 
    driver.find_element_by_xpath('//span[@class="btn_cancel"]').click()

### Enter Details to search in Joonggonara

In [5]:
def get_idx_of_selected_option(option_list, input_text):
    for idx, option in enumerate(option_list):
        if option.text == input_text:
            return idx
    return -1

In [6]:
def enter_details(driver, search_option, item, detail_option):    
    # show 50 items
    driver.find_element_by_xpath('//div[@id="listSizeSelectDiv"]').click()
    time.sleep(1)
    listSize_list = driver.find_elements_by_xpath('//div[@id="listSizeSelectDiv"]/ul/li')
    listSize_list[-1].click()
    time.sleep(1)
    
    # set search options
    driver.find_element_by_xpath('//div[@id="searchOptionSelectDiv"]').click()
    searchBoard_list = driver.find_elements_by_xpath('//div[@id="searchOptionSelectDiv"]/ul/li')
    searchBoard_list[get_idx_of_selected_option(searchBoard_list, search_option['board'])].click()
    time.sleep(1)
    
    driver.find_element_by_xpath('//div[@id="divSearchDateTop"]').click()
    searchDate_list = driver.find_elements_by_xpath('//div[@id="divSearchDateTop"]/ul/li')
    searchDate_list[get_idx_of_selected_option(searchDate_list, search_option['period'])].click()
    
    driver.find_element_by_xpath('//div[@id="divSearchMenuTop"]').click()
    searchMenu_list = driver.find_elements_by_xpath('//div[@id="divSearchMenuTop"]/ul/li')
    searchMenu_list[get_idx_of_selected_option(searchMenu_list, search_option['menu'])].click()
    
    driver.find_element_by_xpath('//div[@id="divSearchByTop"]').click()
    searchBy_list = driver.find_elements_by_xpath('//div[@id="divSearchByTop"]/ul/li')
    searchBy_list[get_idx_of_selected_option(searchBy_list, search_option['by'])].click()
    
    # enter item
    driver.find_element_by_xpath('//input[@placeholder="검색어를 입력해주세요"]').send_keys(item)
    
    # click detail search btn
    driver.find_element_by_xpath('//*[@id="detailSearchBtn"]').click()

    # enter detail options
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 모두 포함"]').send_keys(detail_option['keywords_and'])
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 제외"]').send_keys(detail_option['keywords_not'])
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 중 1개 이상 포함"]').send_keys(detail_option['keywords_or'])
    driver.find_element_by_xpath('//input[@placeholder="다음 어절, 어구 정확히 일치"]').send_keys(detail_option['sentence'])
    
    
    # search
    driver.find_element_by_xpath('//form[@name="frmSearchTop"]/div[@class="input_search_area"]/button[@class="btn-search-green"]').click()
    
    time.sleep(1)

### Get Product Info

In [7]:
def get_post_info(driver, href):
    driver.get(href)
    time.sleep(1)
    driver.switch_to.frame('cafe_main')
    soup = bs(driver.page_source, 'html.parser')

    # post_title
    title = soup.select('div.tit-box span.b')[0].get_text()
    
    # posted_cost
    try:
        cost = soup.select('span.cost')[0].get_text()
    except:
        cost = 0

    # merge contents to single text
    content_tags = soup.select('#tbody')[0].select('p')
    content = ' '.join([ tags.get_text() for tags in content_tags ])
    
    time.sleep(1)

    return {'cost':cost, 'title' : title, 'content' : content}

#### Run ChromeDriver

In [8]:
driver = webdriver.Chrome()
driver.implicitly_wait(2)

#### Login Naver

In [9]:
user_id, user_pw = load_user_info()
login_naver(driver, user_id, user_pw)

#### Search items from Joonggonara and get boards

In [10]:
# log onto Joonggonara and enter details
driver.get('https://cafe.naver.com/joonggonara?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10050146%26search.searchBy=0')
time.sleep(1)
driver.switch_to.frame(driver.find_element_by_name("cafe_main"))

search_option={
    'board' : "게시글 전체",
    'period' : "전체기간",
    'menu' : "주변기기/악세사리",
    'by' : "제목만"
}

item = "에어팟"

detail_option = {
    'keywords_and' : "미개봉 무선 에어팟 2",
    'keywords_not' : "중고폰 유선 삽니다 프로 사요 구매 구함 구해요 구합니다", 
    'keywords_or' : "", 
    'sentence' : ""
}

num_of_items=500

enter_details(driver, search_option, item, detail_option)

In [11]:
board_navigator = driver.find_element_by_xpath('//div[@class="prev-next"]')
board_list = board_navigator.find_elements_by_tag_name('a')
board_href = board_list[0].get_attribute('href')[:-1]
board_idx = 0

#### Get Posts Links

In [12]:
post_key_list=[]
author_list=set([])
while True:
    # change board
    board_idx+=1
    driver.get(board_href+str(board_idx))
    time.sleep(1)
    driver.switch_to.frame(driver.find_element_by_name("cafe_main"))
    
    # get posts in current board
    posts = driver.find_elements_by_css_selector('div.article-board > table > tbody > tr')
    num_of_new_posts = len(posts)
    
    for post in posts:
        
        # get valid posts
        try:
            author = post.find_element_by_class_name('td_name').text.strip()
            href = post.find_element_by_class_name('article').get_attribute('href')
        except:
            num_of_new_posts-=1
            continue
            
        # filter duplicated postings by author
        if author in author_list:
            num_of_new_posts-=1
            continue
        post_key_list.append({"author": author, "href":href})
        author_list.add(author)
        
    # check
    print("read {} pages {} posts. and {} new posts".format(board_idx, len(post_key_list), num_of_new_posts))
    
    # terminate condition
    if num_of_new_posts == 0 or len(post_key_list)>num_of_items:
        break

read 1 pages 34 posts. and 34 new posts
read 2 pages 73 posts. and 39 new posts
read 3 pages 108 posts. and 35 new posts
read 4 pages 141 posts. and 33 new posts
read 5 pages 162 posts. and 21 new posts
read 6 pages 184 posts. and 22 new posts
read 7 pages 207 posts. and 23 new posts
read 8 pages 230 posts. and 23 new posts
read 9 pages 255 posts. and 25 new posts
read 10 pages 272 posts. and 17 new posts
read 11 pages 283 posts. and 11 new posts
read 12 pages 293 posts. and 10 new posts
read 13 pages 303 posts. and 10 new posts
read 14 pages 303 posts. and 0 new posts


#### Get Price Info

In [13]:
price_info=[]
for post_key in post_key_list:
    try:
        post_info = get_post_info(driver, post_key["href"])
        print(post_info['cost'], post_key["author"], post_info['title'])
        price_info.append(post_info)
    except:
        print("cannot attach post")

price_info_pd=pd.DataFrame(price_info)
price_info_pd.to_csv("20200121_무선_"+item+"_price_info.csv")

160,000원 김더기1 에어팟2 무선 미개봉 강남쪽 직거래
168,000원 1등거래지니 에어팟2세대 무선 미개봉 정품 판매합니다 인천주안직거래
170,000원 엘르 에어팟2 무선 미개봉(+무선충전패드) 충주. 서울직 
170,000원 ljkyung 에어팟 2세대 무선충전 미개봉상품 팔아요
185,000원 흰둥이거북이 [판매] 에어팟2세대 무선충전 미개봉(택포)
320,000원 행복한미래이룸 [최저가💚공식스토어] 애플 에어팟 프로 블루투스 무선 이어폰 3세대 한국정품💚미개봉 새제품💚케이스&스티커 증정💦한정 판매중 [서울직거래]
160,000원 무일푼꽃거지 ********국내 정품 에어팟2 무선 비닐 미개봉 새상품 팝니다***************
140,000원 cg9 ●●오랜만에 컴백 에어팟2 유선 무선 에어팟프로 미개봉 10대 팝니다 
2,222원 DdongBBang ※  에어팟2세대  유/무선 미개봉 새상품 판매합니다
170,000원 블리블리캬컁 에어팟2세대/무선충전가능/미개봉/택포
150,000원 smileboyhym 서울 직거래 미개봉 에어팟 유선  무선 한대씩 팝니다
170,000원 1111231 [부산] 에어팟 2세대 무선충전 미개봉
160,000원 쭈니 애플 에어팟 2세대 무선충전 미개봉 새제품 판매합니다.
5,555원 마쉐라람버르 에어팟프로28만원급처합니다 미개봉 새재품 에어팟2 무선 15만원 급처판매요 미개봉
2,000원 꼬롱내 아이폰&에어팟 무선 충전기 "POWER CUBE X2" 미개봉 새상품 5만원에 팝니다
153,000원 매일mae일 에어팟2 무선 미개봉 (2019-8)
150,000원 smurf82 에어팟 2세대 무선 충전 미개봉 새제품 판매합니다 15만원
170,000원 고수를찾아서 에어팟 2세대 무선충전 미개봉 판매합니다
65,000원 그린7522 에어팟2 무선 충전 케이스 정품 미개봉 팝니다
150,000원 auddms95 에어팟2 무선 미개봉 
199,000원 방바리ㅎ 에어팟2세대 유선/무선 미개봉 새상품 팔아요 
1,111,

170,000원 해피데이512 (미개봉)에어팟 2세대 유선,무선
180,000원 mimi80 미개봉 에어팟 2세대 무선 팔아요!!
180,000원 팔슈륨예거 애플 에어팟 2세대 무선충전 미개봉 새상품 팝니다
198,000원 1나공산당 에어팟2 무선충전 미개봉
175,000원 qkrrkdwn12 에어팟2세대 무선충전 모델 미개봉 새제품 판매합니다
175,000원 Sienna L 애플 에어팟 2세대무선충전 정품 미개봉 상품
180,000원 방2222 에어팟2 무선 충전 (미개봉)
230,000원 sunny0627 에어팟 2세대 미개봉 (무선)
200,000원 Levisman 에어팟2 무선충전 미개봉
11,111,111원 정발발 울산 직거래) 에어팟 2세대 유,무선 미개봉 암거나 구매합니다.
2,000원 고독한심야식당 에어팟2세대 무선모델 정품 미개봉 판매합니다 
150,000원 shinra15 울산 미개봉 에어팟/에어팟2/유선충전/무선충전/애플 에어팟
170,000원 감자감자슈감자 에어팟2 무선버전 미개봉 상품
185,000원 조낸달려라 에어팟2  무선충전 미개봉팝니다
185,000원 RkRhd [판매완료]Airpods 에어팟 2세대 무선충전 미개봉 팝니다
170,000원 rlaehdgks82 에어팟2 무선 미개봉세거삽니다
190,000원 equus32 에어팟2세대 무선 미개봉 19만
190,000원 ㄹㅊ 에어팟2 무선충전 미개봉
180,000원 폐지수거 에어팟2 무선 미개봉 새상품
180,000원 totorokin 미개봉 에어팟 2세대 (무선/북미)
230,000원 ㄱ찬희 [미개봉]에어팟 2세대 무선충전 미개봉제품 팝니다
215,000원 coolmannn 에어팟2 무선미개봉팜
2,000원 동막처늘 (판매완료)에어팟 2세대 무선충전 모델 미개봉
27,000원 용식이 AIR i12 (TWS) 블루투스 무선 이어폰 에어팟 에어로팟 차이팟 미개봉 팝니다.
170,000원 이오정보 애플 코리아 국내사양 미개봉 정품 에어팟2세대 유선 30개, 무선 30개 일괄 팝니다.
2